In [ ]:
# stdlib
import os

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
# os.environ["CLUSTER_HTTP_PORT_LOW"] = "9083"

In [ ]:
# stdlib

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
low_port = os.environ.get("CLUSTER_HTTP_PORT_LOW", "auto")
print(environment, low_port)

In [ ]:
# third party
import pandas as pd

# syft absolute
import syft as sy
from syft import test_settings

# Only low side server and login as DS

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port=low_port,
)

In [ ]:
ds_client = server_low.login(
    email="data_scientist@openmined.org", password="verysecurepassword"
)

# Low side research

In [ ]:
# Find available endpoints
ds_client.custom_api.api_endpoints()

In [ ]:
assert len(ds_client.custom_api.api_endpoints()) == 3

# Test the schema

In [ ]:
res = ds_client.api.services.bigquery.schema()
res.get()

In [ ]:
assert isinstance(res.get(), pd.DataFrame)

In [ ]:
# Load these variables for testing but they ideally come from the schema
dataset_1 = test_settings.get("dataset_1", default="dataset_1")
dataset_2 = test_settings.get("dataset_2", default="dataset_2")
table_1 = test_settings.get("table_1", default="table_1")
table_2 = test_settings.get("table_2", default="table_2")
table_2_col_id = test_settings.get("table_2_col_id", default="table_id")
table_2_col_score = test_settings.get("table_2_col_score", default="colname")

____

## Test the `bigquery.test_query` endpoint

In [ ]:
result = ds_client.api.services.bigquery.test_query.mock(
    sql_query=f"SELECT * from {dataset_2}.{table_2} limit 10"
)
assert len(result) == 10

In [ ]:
with sy.raises(sy.SyftException, show=True):
    ds_client.api.services.bigquery.test_query.private(
        sql_query=f"SELECT * from {dataset_2}.{table_2} limit 10"
    )

In [ ]:
LARGE_SAMPLE_QUERY = f"SELECT * FROM {dataset_2}.{table_2} LIMIT 10000"

In [ ]:
mock_res = ds_client.api.services.bigquery.test_query(sql_query=LARGE_SAMPLE_QUERY)

In [ ]:
assert len(mock_res) == 10000

# Submit a query

In [ ]:
FUNC_NAME = "large_sample"

In [ ]:
submission = ds_client.api.services.bigquery.submit_query(
    func_name=FUNC_NAME, query=LARGE_SAMPLE_QUERY
)

In [ ]:
submission

In [ ]:
# todo: this is very noisy, but it actually passes
with sy.raises(
    sy.SyftException(
        public_message="*Please wait for the admin to allow the execution of this code*"
    )
):
    ds_client.code.large_sample(blocking=False)

In [ ]:
with sy.raises(sy.SyftException(public_message="*Your code is waiting for approval*")):
    ds_client.code.large_sample()

# Shutdown

In [ ]:
# todo: add email server shutdown

In [ ]:
if environment != "remote":
    server_low.land()